In [ ]:
import pandas as pd
import vectorbt as vbt
import numpy as np
import talib
import openpyxl

In [ ]:
#get the data
df= pd.read_csv('EURUSD_M15.csv',sep='\t', engine = 'python')
#set time as index

time = df['Time']
df=df.set_index('Time')
trend= df['Close'].to_frame()
trend['trend']=np.nan
def combine_indicators(trend, fastk_period=14, fastd_period=6, SLOWMA=960, FASTMA=8,atr_period=14,
                      Breakeven_no_Xatr=3,tp_no_Xatr=3):
    trend
    fastk_period=14 
    fastd_period=6 
    SLOWMA=960
    FASTMA=8
    atr_period=14 
    Breakeven_no_Xatr=3 
    tp_no_Xatr=3
    #get all indicators

    STOCHF = vbt.IndicatorFactory.from_talib('STOCHF')
    STOCHD=STOCHF.run(df['High'],df['Low'], df['Close'], fastk_period,fastd_period,0 )
    STOCHK=STOCHF.run(df['High'],df['Low'], df['Close'], fastk_period,fastd_period,0 )
    ATR = vbt.IndicatorFactory.from_talib('ATR')
    atr =ATR.run(df['High'], df['Low'], df['Close'], atr_period)
    MA= vbt.IndicatorFactory.from_talib('MA')
    ma = MA.run(df['Close'], SLOWMA,0)

    FA_MA= vbt.IndicatorFactory.from_talib('MA')
    fa_ma = FA_MA.run(df['Close'], FASTMA,0)

    trend['ma'] = ma.real
    trend['fa_ma']= fa_ma.real
    trend['STOCHD']=STOCHD.fastd
    trend['STOCHK']=STOCHK.fastk

    tp_breakeven= True 
    trend['atr']=atr.real
    trend['tp']=np.nan
    trend['tp']=tp_no_Xatr* trend['atr']

    trend['tp2']=np.nan


    trend['sl']=Breakeven_no_Xatr*atr.real
    trend['sl2']= (abs(trend['sl']-df['Close'])/df['Close'])*100
    trend.loc[(trend['STOCHD']<40) & (trend['STOCHD']>trend['STOCHK']) & (trend['Close']> trend['ma']), 'trend']=1
    trend.loc[(trend['STOCHD']>70) & (trend['STOCHD']<trend['STOCHK']) & (trend['Close'] < trend['ma']), 'trend']=2

    # buy=1
    # sell=2

    #converting to percentage
    trend['tp2']=abs(((trend['tp'] - df['Close'])/ (df['Close'])))*100

    trend['breakeven_close']=np.nan
    trend.loc[(trend['trend']==1)|(trend['trend']==2), 'breakeven_close']=df['Close']
    trend['breakeven_close']=trend['breakeven_close'].ffill()


    entries=trend['trend']==1
    trend['exits']= trend.loc[((trend['fa_ma']>trend['breakeven_close'])& (trend['fa_ma'].shift(1)<trend['breakeven_close'])), 'fa_ma'] 
    trend['exits']=trend['exits']>(Breakeven_no_Xatr*trend['atr'])

    short_entries = trend['trend']==2
    trend['short_exits'] =trend.loc[((trend['fa_ma']<trend['breakeven_close'])& (trend['fa_ma'].shift(1)>trend['breakeven_close'])), 'fa_ma'] 
    trend['short_exits']=trend['short_exits']>(Breakeven_no_Xatr*trend['atr'])
    short_exits=trend['short_exits']
    exits = trend['exits']
    return trend


